## Importing Dependencies

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import pandas_datareader.data as web

## Data Augmentation

As we saw last time, the number of expected rows $(1280 \times 2517 = 3221760)$ did not quite match the data we got. This is due to the limitations of the Yahoo Finance API. Thus, we augment the data by using a different source.

In [2]:
# Dropping Adjusted Close to account for a more
# consistent dataset.

df = pd.read_csv("10_Year_Historical.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [3]:
# Filtering tickers with missing data

aapl_data = df[df['Ticker'] == 'AAPL']
standard_length = len(aapl_data)
ticker_value_counts = df['Ticker'].value_counts()
mask = ticker_value_counts < standard_length
tickers_missing_data = ticker_value_counts[mask].index.tolist()

In [4]:
def augment_data(ticker, start_date, end_date):
    try:
        data = web.DataReader(ticker, 'stooq', start_date, end_date)
        data['Ticker'] = ticker
        data['Adj Close'] = data['Close']
        return data[['Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

In [5]:
augmented_dfs = []

with tqdm(total=len(tickers_missing_data), desc='Augmenting data', unit=' stocks') as pbar:
    for ticker in tickers_missing_data:
        missing_dates = aapl_data.index.difference(df[df['Ticker'] == ticker].index)
        start_date, end_date = missing_dates.min(), missing_dates.max()
        augmented = augment_data(ticker, start_date, end_date)
        augmented_dfs.append(augmented)
    
        pbar.update(1)

Augmenting data:   0%|          | 0/2 [00:00<?, ? stocks/s]

In [6]:
df_augmented = pd.concat(augmented_dfs)
df = pd.concat([df, df_augmented])
df.sort_values(by=['Ticker', 'Date'], inplace=True)

In [7]:
print(df.shape)
df.head()

(3221760, 7)


,Ticker,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2014-10-20,AAL,34.189999,35.389999,34.000000,35.220001,33.287529,13532700
2014-10-21,AAL,36.139999,37.880001,36.020000,37.849998,35.773212,26347700
2014-10-22,AAL,38.049999,38.220001,36.939999,37.040001,35.007656,18958400
2014-10-23,AAL,37.430000,40.180000,36.799999,38.480000,36.368645,33292000
2014-10-24,AAL,38.599998,39.950001,38.580002,39.820000,37.635128,17834100


In [8]:
# Check for date mismatches across all tickers

aapl_dates = aapl_data.index.unique()

def check_dates(ticker_data):
    return (ticker_data.index.unique() == aapl_dates).all()

tickers_mismatched_dates = df.groupby('Ticker').apply(
                                            lambda x: not check_dates(x))

mismatched_tickers = tickers_mismatched_dates[tickers_mismatched_dates].index.tolist()
print(mismatched_tickers)

[]


/tmp/ipykernel_2998682/105174469.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tickers_mismatched_dates = df.groupby('Ticker').apply(


In [27]:
df.to_csv('10_Year_Historical_Preprocessed.csv')